# TextRank 
## 핵심단어/문장 구하기

### 2. 핵심 문장 구하기

2-1. 파일 읽기 <br>
2-2. 단어 tokenize 후, 단어 index 만들기<br>
2-3. 문장 그래프를 만들기<br>
2-4. 문장 그래프을 pagerank에 적용하기<br>


In [1]:
import numpy as np
import math
from sklearn.preprocessing import normalize
from collections import Counter
from konlpy.tag import Kkma
from scipy.sparse import csr_matrix

#### 2-1. 파일 읽기

In [2]:
with open('./test1.txt', encoding='utf-8') as f:
    sents = [sent for row in f for sent in row.strip().split(". ") if len(sent) > 0 ]

In [3]:
sents

['인공신경망(artificial neural network, ANN)은 기계학습과 인지과학에서 생물학의 신경망에서 영감을 얻은 통계학적 학습 알고리즘으로 시냅스의 결합으로 네트워크를 형성한 인공 뉴런(노드)이 학습을 통해 시냅스의 결합 세기를 변화시켜, 문제 해결 능력을 가지는 모델 전반을 가리킨다',
 '인공신경망에는 교사 신호(정답)의 입력에 의해서 문제에 최적화되어 가는 교사 학습과 교사 신호를 필요로 하지 않는 비교사 학습이 있다',
 '명확한 해답이 있는 경우에는 교사 학습이, 데이터 클러스터링에는 비교사 학습이 이용된다',
 '다른 기계학습과 같이 신경망은 일반적으로 규칙기반 프로그래밍으로 풀기 어려운 컴퓨터 비전 또는 음성 인식과 같은 다양한 범위의 문제를 푸는데 이용된다.']

#### 2-2. 단어 tokenize 후, 단어 index 만들기

In [7]:
kkma = Kkma()
def kkma_tokenize(sent):
    words = kkma.pos(sent)
    return [word for word, pos in words 
             if ( 'NN' in pos or '/VA' in pos or '/VV' in pos)]

In [15]:
word_counter = Counter(w for sent in sents for w in kkma_tokenize(sent))
print(word_counter)

Counter({'학습': 8, '교사': 6, '신경망': 4, '인공': 3, '문제': 3, '기계': 2, '시냅스': 2, '결합': 2, '신호': 2, '비': 2, '이용': 2, '은': 1, '인지': 1, '과학': 1, '생물학': 1, '영감': 1, '통계': 1, '학적': 1, '알고리즘': 1, '네트워크': 1, '형성': 1, '뉴런': 1, '노드': 1, '이': 1, '세기': 1, '변화': 1, '해결': 1, '능력': 1, '모델': 1, '전반': 1, '정답': 1, '의': 1, '입력': 1, '필요': 1, '해답': 1, '경우': 1, '데이터': 1, '러': 1, '스터링': 1, '일반적': 1, '규칙': 1, '기반': 1, '프로그래밍': 1, '컴퓨터': 1, '비전': 1, '음성': 1, '인식': 1, '다양': 1, '범위': 1})


In [16]:
min_count=2 # Minumum term frequency
min_word_counter = {w:c for w,c in word_counter.items() if c >= min_count}
print(min_word_counter)

{'인공': 3, '신경망': 4, '기계': 2, '학습': 8, '시냅스': 2, '결합': 2, '문제': 3, '교사': 6, '신호': 2, '비': 2, '이용': 2}


In [17]:
## idx_to_vocab -> data type : list
## Word list corresponding row and column
idx_to_vocab = [w for w, _ in sorted(min_word_counter.items(), key=lambda x:-x[1])] 
print("정렬 확인 >> ", sorted(min_word_counter.items(), key=lambda x:-x[1])) # 1번째 요소로 내림차순(-) 정렬
print()
print("정렬을 기준으로 단어 list 생성 >> ", idx_to_vocab)

정렬 확인 >>  [('학습', 8), ('교사', 6), ('신경망', 4), ('인공', 3), ('문제', 3), ('기계', 2), ('시냅스', 2), ('결합', 2), ('신호', 2), ('비', 2), ('이용', 2)]

정렬을 기준으로 단어 list 생성 >>  ['학습', '교사', '신경망', '인공', '문제', '기계', '시냅스', '결합', '신호', '비', '이용']


In [18]:
## vocab_to_idx -> data type : dict
## Vocabulary to index mapper
vocab_to_idx = {vocab:idx for idx, vocab in enumerate(idx_to_vocab)}
print("빈도 순으로 정렬한 list에 번호를 매겨 index 생성\n", vocab_to_idx)  

빈도 순으로 정렬한 list에 번호를 매겨 index 생성
 {'학습': 0, '교사': 1, '신경망': 2, '인공': 3, '문제': 4, '기계': 5, '시냅스': 6, '결합': 7, '신호': 8, '비': 9, '이용': 10}


#### 2-3. 문장 그래프를 만들기

최소 출현 빈도를 넘은 단어들을 이용해 문장 간 similarity를 구한다.<br>
문장 간 similarity가 0.3 보다 작은 경우에는 edge를 연결하지 않는다.

In [204]:
def sent_graph(sents, similarity, min_count=2, min_sim=0.3):
#     _, vocab_to_idx = scan_vocabulary(sents, tokenize, min_count)

    tokens = [[w for w in kkma_tokenize(sent) if w in vocab_to_idx] for sent in sents]
#     tokens = [[w for w in tokenize(sent) if w in vocab_to_idx] for sent in sents]
    rows, cols, data = [], [], []
    n_sents = len(tokens)
    for i, tokens_i in enumerate(tokens):
        for j, tokens_j in enumerate(tokens):
#             if i >= j:
#                 continue
            sim = similarity(tokens_i, tokens_j)
            print("\nS1-S2 similarity = ", sim)
            print("-"*100)
            
            if sim < min_sim:
                continue
            rows.append(i)
            cols.append(j)
            data.append(sim)
    return csr_matrix((data, (rows, cols)), shape=(n_sents, n_sents))


##### cosine similarity 를 사용한 similarity 구하는 방법<br>
짧은 문장에 민감하게 반응한다. <br>
문장에 쓰인 단어가 적은데 비해 동시에 등장한 단어가 여러개라면 유사도가 높게 나올 것이다.

$cos(\theta)=\frac{{A}\cdot{B}}{\|A\|\|B\|}=\frac{\Sigma^{n}_{i=1}{(A_{i})*(B_{i})}}{\sqrt{\Sigma^{n}_{i=1}{(A_{i})^2}}*\sqrt{\Sigma^{n}_{i=1}{(B_{i})^2}}}$

In [205]:
def cosine_sent_sim(s1, s2):
    if (not s1) or (not s2):
        return 0
    
    print("[s1]", s1)
    print("[s2]", s2, "\n")
    
    s1 = Counter(s1)
    s2 = Counter(s2)
    
    print("[s1_counter]", s1)
    print("[s2_counter]", s2, "\n")
    
    norm1 = math.sqrt(sum(v ** 2 for v in s1.values()))
    norm2 = math.sqrt(sum(v ** 2 for v in s2.values()))
    prod = 0
    for k, v in s1.items():
        prod += v * s2.get(k, 0)
        print("[s1_word] %3s \t[s1_word_count]%2d \t[s2.get(s1_word)] %2d \t[prod] %3d"%( k, v, s2.get(k, 0), prod))

    return prod / (norm1 * norm2)

In [206]:
sent_cosine_graph = sent_graph(sents, cosine_sent_sim)

[s1] ['인공', '신경망', '기계', '학습', '신경망', '학습', '시냅스', '결합', '인공', '학습', '시냅스', '결합', '문제']
[s2] ['인공', '신경망', '기계', '학습', '신경망', '학습', '시냅스', '결합', '인공', '학습', '시냅스', '결합', '문제'] 

[s1_counter] Counter({'학습': 3, '인공': 2, '신경망': 2, '시냅스': 2, '결합': 2, '기계': 1, '문제': 1})
[s2_counter] Counter({'학습': 3, '인공': 2, '신경망': 2, '시냅스': 2, '결합': 2, '기계': 1, '문제': 1}) 

[s1_word]  인공 	[s1_word_count] 2 	[s2.get(s1_word)]  2 	[prod]   4
[s1_word] 신경망 	[s1_word_count] 2 	[s2.get(s1_word)]  2 	[prod]   8
[s1_word]  기계 	[s1_word_count] 1 	[s2.get(s1_word)]  1 	[prod]   9
[s1_word]  학습 	[s1_word_count] 3 	[s2.get(s1_word)]  3 	[prod]  18
[s1_word] 시냅스 	[s1_word_count] 2 	[s2.get(s1_word)]  2 	[prod]  22
[s1_word]  결합 	[s1_word_count] 2 	[s2.get(s1_word)]  2 	[prod]  26
[s1_word]  문제 	[s1_word_count] 1 	[s2.get(s1_word)]  1 	[prod]  27

S1-S2 similarity =  1.0
----------------------------------------------------------------------------------------------------
[s1] ['인공', '신경망', '기계', '학습', '신경망', '학습', '시냅스'

In [203]:
print(sent_cosine_graph.todense())

[[1.         0.40006613 0.36514837 0.60246408]
 [0.40006613 1.         0.7768986  0.3380617 ]
 [0.36514837 0.7768986  1.         0.42426407]
 [0.60246408 0.3380617  0.42426407 1.        ]]


##### TextRank 에서 제안한 similarity 구하는 방법

두 문장의 단어 수와 공통으로 등장하는 단어 수를 고려한다.<br>
cosine similarity와 달리 최대 값이 1이 아니고, 문장의 길이가 길수록 높은 유사도를 갖는다.

$sim(s_{1}, s_{2}) = \frac{|{w_{k}}|{w_{k}}\in{S_{1}} \& {w_{k}}\in{S_{2}} |}{\log{|S_{1}|}+\log{|S_{2}|}}$

$𝑆_1$ 문장 1의 단어 수<br>
$𝑆_2$ 문장 2의 단어 수<br>
$|{w_{k}}|{w_{k}}\in{S_{1}} \& {w_{k}}\in{S_{2}}|$ 문장 1과 2에 공통으로 등장한 단어 수<br>


In [213]:
def textrank_sent_sim(s1, s2):
    
    print("\n[s1]", s1)
    print("[s2]", s2)
    
    n1 = len(s1)
    n2 = len(s2)
    if (n1 <= 1) or (n2 <= 1):
        print("???")
        return 0
    
    common = len(set(s1).intersection(set(s2)))
    base = math.log(n1) + math.log(n2)
    
    print("\nS1,S2 교집합 {} --> {}".format(set(s1).intersection(set(s2)), common))
    
    return common / base

In [214]:
sent_textrank_graph = sent_graph(sents, textrank_sent_sim)


[s1] ['인공', '신경망', '기계', '학습', '신경망', '학습', '시냅스', '결합', '인공', '학습', '시냅스', '결합', '문제']
[s2] ['인공', '신경망', '기계', '학습', '신경망', '학습', '시냅스', '결합', '인공', '학습', '시냅스', '결합', '문제']

S1,S2 교집합 {'문제', '신경망', '인공', '결합', '학습', '기계', '시냅스'} --> 7

S1-S2 similarity =  1.36454935837948
----------------------------------------------------------------------------------------------------

[s1] ['인공', '신경망', '기계', '학습', '신경망', '학습', '시냅스', '결합', '인공', '학습', '시냅스', '결합', '문제']
[s2] ['인공', '신경망', '교사', '신호', '문제', '교사', '학습', '교사', '신호', '비', '교사', '학습']

S1,S2 교집합 {'인공', '문제', '학습', '신경망'} --> 4

S1-S2 similarity =  0.7921017934486901
----------------------------------------------------------------------------------------------------

[s1] ['인공', '신경망', '기계', '학습', '신경망', '학습', '시냅스', '결합', '인공', '학습', '시냅스', '결합', '문제']
[s2] ['교사', '학습', '비', '교사', '학습', '이용']

S1,S2 교집합 {'학습'} --> 1

S1-S2 similarity =  0.22953106112437666
----------------------------------------------------------------------------

문장 간 similarity가 0.3 보다 작은 경우에는 edge를 연결하지 않기로 했으므로
1번째 문장과 3번쨰 문장의 유사도는 0이 되었다.

In [215]:
print(sent_textrank_graph.todense())

[[1.36454936 0.79210179 0.         0.95822446]
 [0.79210179 1.40850362 0.70148099 0.73271801]
 [0.         0.70148099 1.11622125 0.58802821]
 [0.95822446 0.73271801 0.58802821 1.55333734]]


#### 1-4. 문장 그래프을 pagerank에 적용하기

##### PageRank

$PR(u)=c*\Sigma_{v∈B_{u}}\frac{PR(v)}{N_{v}}+(1-c)$ <br>

$B_{u}$ 는 page $u$ 의 backlinks 의 출발점 마디이고, $v$ -> $u$로 연결이 있다. <br>
$N_{v}$ 는 $v$가 가진 link 의 개수<br>
$c$는 dampling factor 로 그래프의 다른 vertex로 이동할 확률, $1-c$는 새롭게 vertex에 유입될 확률 <br>



In [25]:
def pagerank(x, df=0.85, max_iter=30, bias=None):
    """
    Arguments
    ---------
    x : scipy.sparse.csr_matrix
        shape = (n vertex, n vertex)
    df : float
        Damping factor, 0 < df < 1
    max_iter : int
        Maximum number of iteration
    bias : numpy.ndarray or None
        If None, equal bias

    Returns
    -------
    R : numpy.ndarray
        PageRank vector. shape = (n vertex, 1)
    """

    assert 0 < df < 1

    # initialize
    A = normalize(x, axis=0, norm='l1')
    R = np.ones(A.shape[0]).reshape(-1,1)

    # check bias
    if bias is None:
        bias = (1 - df) * np.ones(A.shape[0]).reshape(-1,1)
    else:
        bias = bias.reshape(-1,1)
        bias = A.shape[0] * bias / bias.sum()
        assert bias.shape[0] == A.shape[0]
        bias = (1 - df) * bias

    # iteration
    for _ in range(max_iter):
        R = df * (A * R) + bias

    return R

cosine similarity와 textrank similarity를 비교한다

In [26]:
# sent_cosine_graph = sent_graph(sents, cosine_sent_sim)

In [216]:
sent_cosine_graph.todense()

matrix([[1.        , 0.40006613, 0.36514837, 0.60246408],
        [0.40006613, 1.        , 0.7768986 , 0.3380617 ],
        [0.36514837, 0.7768986 , 1.        , 0.42426407],
        [0.60246408, 0.3380617 , 0.42426407, 1.        ]])

In [217]:
# sent_textrank_graph = sent_graph(sents, textrank_sent_sim)

In [218]:
sent_textrank_graph.todense()

matrix([[1.36454936, 0.79210179, 0.        , 0.95822446],
        [0.79210179, 1.40850362, 0.70148099, 0.73271801],
        [0.        , 0.70148099, 1.11622125, 0.58802821],
        [0.95822446, 0.73271801, 0.58802821, 1.55333734]])

In [219]:
damping_factor = 0.85
max_iter=30

In [236]:
rank_cosine_keysent = pagerank(sent_cosine_graph, damping_factor, max_iter).reshape(-1)
print(rank_cosine_keysent)

[0.97279733 1.01887586 1.03656275 0.97176406]


In [237]:
rank_textrank_keysent = pagerank(sent_textrank_graph, damping_factor, max_iter).reshape(-1)
print(rank_textrank_keysent)

[0.95370662 1.10302224 0.79396838 1.14930277]


In [259]:
top = 2

idxs_cosine = np.argsort(-rank_cosine_keysent)[:top]
print("TOP {} Sentence Index >> {}\n".format( top, idxs_cosine))

print("< Cosine Similarity Result >")
for i in idxs_cosine:
    print('rank [{:.4}] \nsent#{} {}\n'.format(rank_cosine_keysent[i], i, sents[i]))


TOP 2 Sentence Index >> [2 1]

< Cosine Similarity Result >
rank [1.037] 
sent#2 명확한 해답이 있는 경우에는 교사 학습이, 데이터 클러스터링에는 비교사 학습이 이용된다

rank [1.019] 
sent#1 인공신경망에는 교사 신호(정답)의 입력에 의해서 문제에 최적화되어 가는 교사 학습과 교사 신호를 필요로 하지 않는 비교사 학습이 있다



In [260]:
idxs_textrank = np.argsort(-rank_textrank_keysent)[:top] 
print("TOP {} Sentence Index >> {}\n".format( top, idxs_textrank))

print("< TextRank Similarity Result >")
for i in idxs_textrank:
    print('rank [{:.4}] \nsent#{} {}\n'.format(rank_textrank_keysent[i], i, sents[i]))

TOP 2 Sentence Index >> [3 1]

< TextRank Similarity Result >
rank [1.149] 
sent#3 다른 기계학습과 같이 신경망은 일반적으로 규칙기반 프로그래밍으로 풀기 어려운 컴퓨터 비전 또는 음성 인식과 같은 다양한 범위의 문제를 푸는데 이용된다.

rank [1.103] 
sent#1 인공신경망에는 교사 신호(정답)의 입력에 의해서 문제에 최적화되어 가는 교사 학습과 교사 신호를 필요로 하지 않는 비교사 학습이 있다



긴 문장을 선호하는 TextRank Similarity에 비해 Cosine Similarity 의 결과에 문장에 쓰인 단어가<br>
적지만 동시에 등장한 단어가 많았기 때문에 짧은 문장의 rank 값이 높은 것을 확인할 수 있다.<br>